In [1]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

In [2]:
def detect_objects(video):

    net = cv2.dnn.readNet('cfg/yolov3.weights', 'cfg/yolov3.cfg')
    classes = []
    with open('cfg/coco.names', 'r') as f:
        classes = f.read().splitlines()

    file_path = filedialog.askopenfilename(title="Click on video to detect:", filetypes=(("MP4 files", ".mp4"), ("All files", ".*")))
    if not file_path:
        return

    cap = cv2.VideoCapture(file_path)

    while True:
        ret, frame = cap.read()
        
        if not ret:
            break
        
        height, width, _ = frame.shape
        
        blob = cv2.dnn.blobFromImage(frame, 1/255.0, (224, 224), swapRB=True, crop=False)
        net.setInput(blob)

        output_layers_names = net.getUnconnectedOutLayersNames()
        layer_outputs = net.forward(output_layers_names)
        
        boxes = []
        confidences = []
        class_ids = []
        
        for output in layer_outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        font = cv2.FONT_HERSHEY_PLAIN
        colors = np.random.uniform(0, 255, size=(len(classes), 3))

        if len(indexes) > 0:
            for i in indexes.flatten():
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                confidence = str(round(confidences[i], 2))
                color = colors[i]
                cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                cv2.putText(frame, label + " " + confidence, (x, y + 20), font, 2, (255, 255, 255), 2)

        cv2.imshow('video', frame)
        
        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    #cv2.waitKey(0)

    cap.release()
    cv2.destroyAllWindows()

def start():
    user = user_entry.get()
    if user:
        welcome.config(text="Welcome, " + user + ".")
        detect_objects(user)

root = tk.Tk()
root.title("Object Detection")

welcome = tk.Label(root, text="Welcome")
welcome.pack()
user_label = tk.Label(root, text="Kindly enter your username: ")
user_label.pack()
user_entry = tk.Entry(root)
user_entry.pack()

btn = tk.Button(root, text="Detect Objects", command=start)
btn.pack()

root.mainloop()

2024-04-24 11:17:47.942 python[25100:2613648] +[CATransaction synchronize] called within transaction
2024-04-24 11:17:48.012 python[25100:2613648] +[CATransaction synchronize] called within transaction


: 